In [ ]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from tkinter import Tk, filedialog, Button, Label
from PIL import Image, ImageTk

# Define the BaselineModel class 
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 32 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
# Load the trained model
model = BaselineModel()
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Define transformations to preprocess the input image
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Function to classify the uploaded image
def classify_image():
    file_path = filedialog.askopenfilename(initialdir='/', title='Select an image',
                                           filetypes=[('Image files', ('*.jpg', '*.png', '*.jpeg'))])
    if file_path:
        image = Image.open(file_path)
        image = transform(image).unsqueeze(0)  

        with torch.no_grad():
            output = model(image)
            _, predicted = torch.max(output.data, 1)
            probabilities = torch.softmax(output, dim=1)[0]  # Get probabilities for each class

        class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        classification = class_names[predicted.item()]

        # Calculate the percentage of confidence (probability) for the predicted class
        confidence = probabilities[predicted].item() * 100

        # Update the label with the classification result and confidence percentage
        result_label.config(text=f"Classification: {classification}\nConfidence: {confidence:.2f}%")

        # Display the uploaded image in the UI
        img = Image.open(file_path)
        img.thumbnail((150, 150))  
        img = ImageTk.PhotoImage(img)
        image_label.config(image=img)
        image_label.image = img  

# Create the GUI window
root = Tk()
root.title('Computer Vision Classifier')
root.geometry('400x300')  

# Create a button to upload an image
upload_button = Button(root, text="Upload Image", command=classify_image)
upload_button.pack(pady=20)

# Label to show the uploaded image
image_label = Label(root)
image_label.pack()

# Label to show the classification result
result_label = Label(root, text="")
result_label.pack()

root.mainloop()
